# Wygładzenie danych
Notatnik służący do wygładzenia danych w celu usunięcia nagłych skoków wartości w krótkim czasie.

In [ ]:
import pandas as pd
organised_directory = "Organised"
labels = ['aggregation', 'filtration', 'filtration-aggregation', 'filtration-aggregation-join', 'filtration-join']

def smooth_values(entry_dir, window_value, labels, normalized):
    if normalized:
        path_to_df = f"./../{entry_dir}/Time_series_udf_dataset_normalized.csv"
    else:
        path_to_df = f"./../{entry_dir}/Time_series_udf_dataset.csv"
    df = pd.read_csv(path_to_df)
    full_df = pd.DataFrame()
    for label in labels:
        print(f"Zacznij przetwarzanie {label} dla okna {window_value}")
        partial_df = df.loc[df.label == label].copy().reset_index()[["snapshot", "label", "udf", "epoch", "CPU", "RAM", "size"]]
        cpu_df = partial_df.groupby("snapshot")['CPU'].rolling(window=window_value, min_periods = 1).mean().to_frame()
        ram_df = partial_df.groupby("snapshot")['RAM'].rolling(window=window_value, min_periods = 1).mean().to_frame()

        partial_df['CPU'] = cpu_df.reset_index()['CPU'] 
        partial_df['CPU'] = cpu_df.reset_index()['CPU'].round(2)

        partial_df['RAM'] = ram_df.reset_index()['RAM']
        partial_df['RAM'] = ram_df.reset_index()['RAM'].round(2)


        partial_df['epoch'] = partial_df['epoch'].round(3)
        full_df = pd.concat([full_df, partial_df.fillna(0).reset_index()])
    if normalized:
        full_df[["snapshot", "label", "udf", "epoch", "CPU", "RAM", "size"]].fillna(0).to_csv(f"./../{entry_dir}/Time_series_udf_dataset_normalized_smooth_{window_value}.csv", index = False)
        print(f"Zapisano plik ./../{entry_dir}/Time_series_udf_dataset_normalized_smooth_{window_value}.csv")
    else:
        full_df[["snapshot", "label", "udf", "epoch", "CPU", "RAM", "size"]].fillna(0).to_csv(f"./../{entry_dir}/Time_series_udf_dataset_smooth_{window_value}.csv", index = False)
        print(f"Zapisano plik ./../{entry_dir}/Time_series_udf_dataset_smooth_{window_value}.csv")
    return full_df
    
window_sizes = [6, 12, 18]
for window_size in window_sizes:
    smooth_values(organised_directory, window_size, labels, normalized = False)
    smooth_values(organised_directory, window_size, labels, normalized = True)